** Please run this in Google Colab

In [1]:
!pip install streamlit
!npm install -g localtunnel
# Installations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.678s


In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Comment out these 2 lines ^

Mounted at /content/drive


In [4]:
%%writefile deepfake_detector.py
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

st.title('DeepFake Detection') # app title

# Upload image and show
test_image = st.file_uploader("Upload test image", type=["jpg", "jpeg"])

# Let the user select the model
model_names = {
    'Model 1: EfficientNet B0 ': 'EfficientNetb0_test1.h5',
    'Model 2: EfficientNet B0 with Regularization': 'EfficientNetb0withReg_test2.h5',
    'Model 3: EfficientNet V2 B0': 'EfficientNetv2b0_test3.h5',
    'Model 4: Xception V1': 'Xception_v1_test4.h5'
}

model_option = st.selectbox('Choose the model for prediction:', options=list(model_names.keys()))

# function to load model
@st.cache_resource
def load_model_wrapper(model_filename):
    model_path = f'/content/drive/My Drive/AI/{model_filename}' # --> Comment out this line
    model = load_model(model_path) # --> Comment out this line
    # model = load_model(model_filename) --> Uncomment this line
    return model

if test_image is not None:
    # Display the uploaded image
    uploaded_image = Image.open(test_image)
    st.image(uploaded_image, caption='Uploaded Image', use_column_width=True)

    # Preprocess the image for the model
    img = uploaded_image.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Load the selected model (only once per session)
    model_filename = model_names[model_option]
    model_selected = load_model_wrapper(model_filename)

    # Make a prediction
    prediction = model_selected.predict(img_array)

    # Convert the prediction to a percentage
    prediction_percentages = prediction[0] * 100

    # Display the prediction probabilities with appropriate labels
    st.write("Prediction Probabilities:")
    col1, col2 = st.columns(2)
    with col1:
        st.metric("Fake Probability", f"{prediction_percentages[0]:.2f}%")
    with col2:
        st.metric("Real Probability", f"{prediction_percentages[1]:.2f}%")


Overwriting deepfake_detector.py


In [5]:
!pip install streamlit
!pip install pyngrok  # Specify the version if there are issues with the latest

# Installations for GUI

In [25]:
!pkill ngrok
# To kill any existing ngrok processes as the free version supports only 1 to run

In [26]:
!streamlit run deepfake_detector.py &>/dev/null &
# Run the streamlit app

In [27]:
from pyngrok import ngrok

# Kill any existing ngrok tunnels
ngrok.kill()
ngrok.set_auth_token('2f5ZxALbg0KZdjk5edZRCg12MVo_6uVhA7hUmottYgcHGANTE')
# Setup a new ngrok tunnel
# Create an ngrok tunnel to port 8501 to connect to the streamlit
public_url = ngrok.connect(addr='8501', proto='http')
print(public_url)



# !streamlit run --server.port 8501 deepfake_detector.py &>/dev/null&


NgrokTunnel: "https://fe5e-34-106-81-233.ngrok-free.app" -> "http://localhost:8501"
